In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/am3052/.conda/envs/elk/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

## Load model and tokenizer

In [4]:
class ModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        
        self.model = model
    
    def forward(self, input_ids, attention_mask):
        return self.model(input_ids, attention_mask)

In [35]:
class RewardModel(nn.Module):
    def __init__(self):
        super().__init__()

        model = AutoModelForCausalLM.from_pretrained(
            "EleutherAI/gpt-j-6b", cache_dir="../../.hf_cache/hub"
        )

        self.transformer = model.transformer
        self.v_head = nn.Linear(model.config.n_embd, 1, bias=False, dtype=torch.float16)
        
        self.PAD_ID = model.config.eos_token_id


    def forward(self, input_ids, attention_mask):
        hidden_states = self.transformer(
            input_ids, attention_mask=attention_mask
        )[0]


        rewards = self.v_head(hidden_states).squeeze(-1)
        
        ends = (input_ids == self.PAD_ID).int().argmax(dim=1, keepdim=True)
        rewards = torch.gather(rewards, 1, ends)
        
        return rewards

Load the model (might take a few minutes):

In [ ]:
state_dict = torch.load(WEIGHTS_PATH)
state_dict

In [36]:
WEIGHTS_PATH = (
    "../../.hf_cache/hub/models--Dahoas--gptj-rm-static/"
    "snapshots/dc9bb2f15f4cddace8a812174c3e7afda2308258/hf_ckpt.pt"
)

reward_model = ModelWrapper(model=RewardModel())
reward_model.load_state_dict(state_dict, strict=False)
reward_model.to(DEVICE)
reward_model.model.v_head.weight.data = reward_model.model.v_head.weight.data.float()
reward_model

ModelWrapper(
  (model): RewardModel(
    (transformer): GPTJModel(
      (wte): Embedding(50400, 4096)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-27): 28 x GPTJBlock(
          (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (attn): GPTJAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (mlp): GPTJMLP(
            (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
            (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
            (act): NewGELUActivation()
            (dropout): Dropo

In [52]:
with torch.no_grad():
    out = reward_model.model(
        input_ids=tokenized_text["chosen"].to(DEVICE),
        attention_mask=tokenized_text["chosen_attention_mask"].to(DEVICE)
    )

out.shape

torch.Size([1, 1])

Load the tokenizer:

In [53]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6b")
tokenizer.pad_token = tokenizer.eos_token

MAX_LENGTH = 1024
EOS_TOKEN = tokenizer.eos_token

## Load the dataset

In [54]:
dataset = load_dataset("Dahoas/rm-static", split="test", cache_dir="../../.hf_cache/datasets")
dataset

Found cached dataset parquet (/rds/user/am3052/hpc-work/elk-rlhf/notebooks/../../.hf_cache/datasets/Dahoas___parquet/default-b9d2c4937d617106/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['prompt', 'response', 'chosen', 'rejected'],
    num_rows: 5103
})

## Tokenize the dataset

In [55]:
def tokenize_row(row, tokenizer, max_length=1024):
    prompt = row["prompt"]
    chosen_response, rejected_response = row["chosen"], row["rejected"]

    # Note that we do not have to truncate in this specific case since
    # all input sequences will be shorter than max_length
    chosen_tokenized = tokenizer(
        prompt + chosen_response + EOS_TOKEN,
        padding="max_length", max_length=max_length, return_tensors="pt",
    )
    rejected_tokenized = tokenizer(
        prompt + rejected_response + EOS_TOKEN,
        padding="max_length", max_length=max_length, return_tensors="pt",
    )

    return {
        "chosen": chosen_tokenized["input_ids"],
        "rejected": rejected_tokenized["input_ids"],
        "chosen_attention_mask": chosen_tokenized["attention_mask"],
        "rejected_attention_mask": rejected_tokenized["attention_mask"],
    }

tokenized_text = tokenize_row(dataset[0], tokenizer)
list(map(lambda x: x.shape[1], tokenized_text.values())), list(tokenized_text.keys()), tokenized_text["chosen"][:, :10]

([1024, 1024, 1024, 1024],
 ['chosen', 'rejected', 'chosen_attention_mask', 'rejected_attention_mask'],
 tensor([[  198,   198, 20490,    25,   314,   716,  2111,   284,  3551,   257]]))

In [56]:
# dataset = dataset.map(tokenize_row, fn_kwargs={ "tokenizer": tokenizer })
# dataset = dataset.remove_columns(["prompt", "response"])
# dataset

## Do inferece

In [68]:
# Define batch size and create a data loader
batch_size = 2
data_loader = DataLoader(dataset, batch_size=batch_size)

# Iterate over batches and perform inference
for i, batch in enumerate(data_loader):
    print(i)
    
    prompts = batch["prompt"]
    chosen_responses, rejected_responses = batch["chosen"], batch["rejected"]
    
    chosen_responses = [
        prompt + chosen_response + EOS_TOKEN for prompt, chosen_response in zip(prompts, chosen_responses)
    ]
    rejected_responses = [
        prompt + rejected_response + EOS_TOKEN for prompt, rejected_response in zip(prompts, rejected_responses)
    ]

    # Note that we do not have to truncate in this specific case since
    # all input sequences will be shorter than max_length
    chosen_tokenized = tokenizer(
        chosen_responses,
        padding="max_length", max_length=MAX_LENGTH, return_tensors="pt",
    )
    rejected_tokenized = tokenizer(
        rejected_responses,
        padding="max_length", max_length=MAX_LENGTH, return_tensors="pt",
    )
    
    input_ids = torch.cat([
        chosen_tokenized["input_ids"], rejected_tokenized["input_ids"]
    ]).to(DEVICE)
    attention_mask = torch.cat([
        chosen_tokenized["attention_mask"], rejected_tokenized["attention_mask"]
    ]).to(DEVICE)
    
    # First half is chosen, other half is rejected
    with torch.no_grad():
        rewards = reward_model.model(
            input_ids=input_ids, attention_mask=attention_mask
        )
    print(rewards.shape)
    break

0


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 79.15 GiB total capacity; 77.50 GiB already allocated; 445.88 MiB free; 77.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF